In [18]:
import os
import numpy as np
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob









In [14]:
MAX_SAMPLES = 2000

image_dir = "/content/drive/MyDrive/1/face_crop"
mask_dir = "/content/drive/MyDrive/1/face_crop_segmentation"

# Load and limit the number of samples
image_paths = sorted(glob.glob(os.path.join(image_dir, "*.jpg")))[:MAX_SAMPLES]
mask_paths = sorted(glob.glob(os.path.join(mask_dir, "*.jpg")))[:MAX_SAMPLES]


In [20]:
from tqdm import tqdm
IMG_HEIGHT, IMG_WIDTH = 128, 128

X, Y = [], []

for img_path, mask_path in tqdm(zip(image_paths, mask_paths), total=len(image_paths)):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))
    mask = (mask > 127).astype(np.uint8)

    X.append(img)
    Y.append(mask)

X = np.array(X) / 255.0
Y = np.expand_dims(np.array(Y), axis=-1)

100%|██████████| 2000/2000 [00:31<00:00, 63.73it/s]


In [21]:


# Train/Validation split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [22]:
# Build U-Net model
def build_unet(input_shape):
    inputs = Input(input_shape)

    # Encoder
    c1 = Conv2D(16, 3, activation='relu', padding='same')(inputs)
    c1 = Conv2D(16, 3, activation='relu', padding='same')(c1)
    p1 = MaxPooling2D()(c1)

    c2 = Conv2D(32, 3, activation='relu', padding='same')(p1)
    c2 = Conv2D(32, 3, activation='relu', padding='same')(c2)
    p2 = MaxPooling2D()(c2)

    # Bottleneck
    b1 = Conv2D(64, 3, activation='relu', padding='same')(p2)
    b1 = Conv2D(64, 3, activation='relu', padding='same')(b1)

    # Decoder
    u1 = UpSampling2D()(b1)
    u1 = concatenate([u1, c2])
    c3 = Conv2D(32, 3, activation='relu', padding='same')(u1)
    c3 = Conv2D(32, 3, activation='relu', padding='same')(c3)

    u2 = UpSampling2D()(c3)
    u2 = concatenate([u2, c1])
    c4 = Conv2D(16, 3, activation='relu', padding='same')(u2)
    c4 = Conv2D(16, 3, activation='relu', padding='same')(c4)

    outputs = Conv2D(1, 1, activation='sigmoid')(c4)

    model = Model(inputs, outputs)
    return model

In [23]:
# Compile and train model
model = build_unet((IMG_HEIGHT, IMG_WIDTH, 3))
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=10,
    batch_size=16,
    verbose=1
)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 245s 2s/step - accuracy: 0.6860 - loss: 0.6027 - val_accuracy: 0.7784 - val_loss: 0.4586
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step - accuracy: 0.7723 - loss: 0.4708 - val_accuracy: 0.8063 - val_loss: 0.4251
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 227s 2s/step - accuracy: 0.7947 - loss: 0.4431 - val_accuracy: 0.8081 - val_loss: 0.4217
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 219s 2s/step - accuracy: 0.8012 - loss: 0.4356 - val_accuracy: 0.8165 - val_loss: 0.4102
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 270s 2s/step - accuracy: 0.8038 - loss: 0.4267 - val_accuracy: 0.8245 - val_loss: 0.3921
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 261s 2s/step - accuracy: 0.8146 - loss: 0.4063 - val_accuracy: 0.8315 - val_loss: 0.3779
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 263s 2s/step - accuracy: 0.8202 - loss: 0.3980 - val_accuracy: 0.8372 - val_loss: 0.3687
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 263s 2s/step - accuracy: 0.8266 - loss: 0.3865 - val_accu

In [26]:

preds = model.predict(X_val)
preds_bin = (preds > 0.5).astype(np.uint8)

13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 995ms/step


In [27]:

def compute_iou(gt, pred):
    intersection = np.logical_and(gt, pred)
    union = np.logical_or(gt, pred)
    return np.sum(intersection) / np.sum(union)

def compute_dice(gt, pred):
    intersection = np.sum(gt * pred)
    return (2. * intersection) / (np.sum(gt) + np.sum(pred))

ious, dices = [], []
for gt, pred in zip(Y_val, preds_bin):
    gt = gt.squeeze()
    pred = pred.squeeze()
    ious.append(compute_iou(gt, pred))
    dices.append(compute_dice(gt, pred))

print(f"U-Net Average IoU: {np.mean(ious):.4f}")
print(f"U-Net Average Dice Score: {np.mean(dices):.4f}")

U-Net Average IoU: 0.6288
U-Net Average Dice Score: 0.7517
